# 2. Data preprocessing
In this script we will:
* remove missing and duplicates
* clean text:
  * remove URL
  * remove @ tags
  * remove numbers
  * remove special signs
  * remove newline characters
  * remove tabs
  * lowercase strings
* tokenize
* remove stopwords

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import re

import gensim
from gensim.utils import simple_preprocess
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
# read labeled data
df = pd.read_csv('/content/drive/MyDrive/ZUM_NLP/tweets_data_labeled.csv', index_col=False)
df.drop(['Unnamed: 0','cleaned_text','sentiment'], axis=1, inplace=True)
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,text,sentiments_val
0,"Kiedy #coronavirus dotrze do Polski, napotka o...",neutral
1,Kolejni Polacy chcą wrócić z Wuhan https://t.c...,positive
2,A tymczasem w Wuhan odcięto od reszty świata o...,negative
3,@PatrykWachowiec @KancelariaSejmu nie zdziwię ...,positive
4,".@MSZ_RP: 19 Polaków chce wrócić z Wuhan, w am...",positive
...,...,...
456913,"@Przedszkolanka4 @MagdaCDN W piątek, 21 maja P...",negative
456914,Przyjdź i porozmawiaj z nami!\n\nDowiedz się w...,neutral
456915,Serio chcą wprowadzić pracę zdalną w wymiarze ...,neutral
456916,Szczepienia przeciw Covid-19 będą organizowane...,negative


In [ ]:
# distribution of sentiments
df.groupby('sentiments_val').nunique()

,text
sentiments_val,
negative,163505
neutral,112645
positive,180767


In [ ]:
# Check if there are missing values or duplicates. If so, we remove them.
print(df.isnull().sum())
df.dropna(inplace=True)

print('Number of duplicates:', df.duplicated().sum())

text              1
sentiments_val    0
dtype: int64
Number of duplicates: 0


In [ ]:
# remove tweet 146838 because this tweet contain only @...
df.drop([146838], axis=0, inplace = True)
df.reset_index(inplace=True)

In [ ]:
# cleaning the data like in previous script but without lematization
def clean_data(text):
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    text = re.sub('\S*@\S*\s?', ' ', text)
    text = re.sub('\$[a-zA-Z0-9]*', ' ', text)
    text = re.sub('\@[a-zA-Z0-9]*', ' ', text)
    text = re.sub('\d+', ' ', text)
    text = re.sub(r'[^\w]', ' ', text)
    text = re.sub("\'", "", text)
    text = re.sub('\t', '', text)
    text = re.sub('\n', ' ', text)
    return text

In [ ]:
# clean data and save to list
data_to_list = []
temp = df['text'].values.tolist()
for i in range(len(temp)):
    data_to_list.append(clean_data(temp[i]))

del temp
list(data_to_list[:5])

['Kiedy  coronavirus dotrze do Polski  napotka opór znakomicie zorganizowanego publicznego systemu ochrony zdrowia  A jeśli nie napotka ',
 'Kolejni Polacy chcą wrócić z Wuhan  ',
 'A tymczasem w Wuhan odcięto od reszty świata obszar porównywalny do Syrii   MCD i Starbucks zawiesili działalność i   ',
 '  nie zdziwię się  jak powiedzą  że listy pojechały do Chin do Wuhan   kto odważny po nie pojedzie  ',
 '   Polaków chce wrócić z Wuhan  w ambasadzie RP w Pekinie powołano sztab kryzysowy  Będzie też spotkanie   ']

In [ ]:
# tokenization with simple preprocess (from gensim module)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data_to_list))

print(*data_words[:10], sep='\n')

['kiedy', 'coronavirus', 'dotrze', 'do', 'polski', 'napotka', 'opor', 'znakomicie', 'zorganizowanego', 'publicznego', 'systemu', 'ochrony', 'zdrowia', 'jesli', 'nie', 'napotka']
['kolejni', 'polacy', 'chca', 'wrocic', 'wuhan']
['tymczasem', 'wuhan', 'odcieto', 'od', 'reszty', 'swiata', 'obszar', 'porownywalny', 'do', 'syrii', 'mcd', 'starbucks', 'zawiesili', 'działalnosc']
['nie', 'zdziwie', 'sie', 'jak', 'powiedza', 'ze', 'listy', 'pojechały', 'do', 'chin', 'do', 'wuhan', 'kto', 'odwazny', 'po', 'nie', 'pojedzie']
['polakow', 'chce', 'wrocic', 'wuhan', 'ambasadzie', 'rp', 'pekinie', 'powołano', 'sztab', 'kryzysowy', 'bedzie', 'tez', 'spotkanie']
['cyklu', 'tego', 'jeszcze', 'nie', 'grano', 'przypadku', 'wuhan', 'ciekawe', 'słowa', 'szczegolnie', 'jesli', 'wezmiemy', 'pod', 'uwage', 'doswiadczen']
['uzyteczni', 'idioci', 'akcji', 'chinski', 'wirus', 'wuhan', 'cyklu', 'stan', 'prawami', 'dla', 'mniejszosci']
['polakow', 'chce', 'wrocic', 'zagrozonego', 'miasta', 'wuhan', 'transport', 'm

In [ ]:
# Read stopwords (dowloaded from github)
!wget https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt

with open("polish.stopwords.txt", "r") as f:
    stop_words = [line.rstrip('\n') for line in f]

--2022-01-04 13:57:58--  https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2016 (2.0K) [text/plain]
Saving to: ‘polish.stopwords.txt.4’

polish.stopwords.tx 100%[===================>]   1.97K  --.-KB/s    in 0s      

2022-01-04 13:57:58 (6.54 MB/s) - ‘polish.stopwords.txt.4’ saved [2016/2016]



In [ ]:
# remove stopwords
for i in range(len(data_words)):
    data_words[i] = [word for word in data_words[i] if word not in stop_words]

print(*data_words[:10], sep='\n')

['coronavirus', 'dotrze', 'polski', 'napotka', 'opor', 'znakomicie', 'zorganizowanego', 'publicznego', 'systemu', 'ochrony', 'zdrowia', 'napotka']
['kolejni', 'polacy', 'chca', 'wrocic', 'wuhan']
['tymczasem', 'wuhan', 'odcieto', 'reszty', 'swiata', 'obszar', 'porownywalny', 'syrii', 'mcd', 'starbucks', 'zawiesili', 'działalnosc']
['zdziwie', 'powiedza', 'listy', 'pojechały', 'chin', 'wuhan', 'odwazny', 'pojedzie']
['polakow', 'chce', 'wrocic', 'wuhan', 'ambasadzie', 'rp', 'pekinie', 'powołano', 'sztab', 'kryzysowy', 'tez', 'spotkanie']
['cyklu', 'grano', 'przypadku', 'wuhan', 'ciekawe', 'słowa', 'szczegolnie', 'wezmiemy', 'uwage', 'doswiadczen']
['uzyteczni', 'idioci', 'akcji', 'chinski', 'wirus', 'wuhan', 'cyklu', 'stan', 'prawami', 'mniejszosci']
['polakow', 'chce', 'wrocic', 'zagrozonego', 'miasta', 'wuhan', 'transport', 'odbywac', 'etapami']
['okaze', 'coronavirus', 'chinski', 'wirus', 'dojdzie', 'polski', 'przynajmniej', 'media', 'odtrabia', 'zos']
['drugiej', 'fazie', 'bezobjawo

In [ ]:
# detokenization
data = []
for i in range(len(data_words)):
    data.append(TreebankWordDetokenizer().detokenize(data_words[i]))

del data_words
print(*data[:10], sep='\n')

coronavirus dotrze polski napotka opor znakomicie zorganizowanego publicznego systemu ochrony zdrowia napotka
kolejni polacy chca wrocic wuhan
tymczasem wuhan odcieto reszty swiata obszar porownywalny syrii mcd starbucks zawiesili działalnosc
zdziwie powiedza listy pojechały chin wuhan odwazny pojedzie
polakow chce wrocic wuhan ambasadzie rp pekinie powołano sztab kryzysowy tez spotkanie
cyklu grano przypadku wuhan ciekawe słowa szczegolnie wezmiemy uwage doswiadczen
uzyteczni idioci akcji chinski wirus wuhan cyklu stan prawami mniejszosci
polakow chce wrocic zagrozonego miasta wuhan transport odbywac etapami
okaze coronavirus chinski wirus dojdzie polski przynajmniej media odtrabia zos
drugiej fazie bezobjawowej coronavirus zaatakowac serce prowadzic arytmi monitor wykaze


In [ ]:
df['text_prep'] = pd.DataFrame(data)
df

,index,text,sentiments_val,text_prep
0,0,"Kiedy #coronavirus dotrze do Polski, napotka o...",neutral,coronavirus dotrze polski napotka opor znakomi...
1,1,Kolejni Polacy chcą wrócić z Wuhan https://t.c...,positive,kolejni polacy chca wrocic wuhan
2,2,A tymczasem w Wuhan odcięto od reszty świata o...,negative,tymczasem wuhan odcieto reszty swiata obszar p...
3,3,@PatrykWachowiec @KancelariaSejmu nie zdziwię ...,positive,zdziwie powiedza listy pojechały chin wuhan od...
4,4,".@MSZ_RP: 19 Polaków chce wrócić z Wuhan, w am...",positive,polakow chce wrocic wuhan ambasadzie rp pekini...
...,...,...,...,...
456911,456913,"@Przedszkolanka4 @MagdaCDN W piątek, 21 maja P...",negative,piatek maja punkt szczepien powszechnych ul no...
456912,456914,Przyjdź i porozmawiaj z nami!\n\nDowiedz się w...,neutral,przyjdz porozmawiaj dowiedz wiecej szczepieniu...
456913,456915,Serio chcą wprowadzić pracę zdalną w wymiarze ...,neutral,serio chca wprowadzic prace zdalna wymiarze dn...
456914,456916,Szczepienia przeciw Covid-19 będą organizowane...,negative,szczepienia przeciw covid organizowane szkołac...


In [ ]:
# save preprocessed data
df.to_csv('/content/drive/MyDrive/ZUM_NLP/tweets_data_labeled_prepro.csv')